In [55]:
# requests is linxu function for getting websites
import requests  

In [56]:
r = requests.get('http://www.joburgmarket.co.za/dailyprices.php')

In [57]:
# print a portion of the files - top 200 characters
print(r.text[0:200])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
	<title> Daily Prices | 


In [58]:
# now that we have the file; use Beautifulsoup to read the file - resolve a file into its subsections
from bs4 import BeautifulSoup  

# if you reading from webpage
soup = BeautifulSoup(r.text, 'html.parser')  

## Get all the commodity information

In [59]:
#This code searches the soup object for all td tags with the attribute class="tleft2". 
#It returns a special Beautiful Soup object (called a "ResultSet") containing the search results.

# Commodity is the name of the object being sold
commodity = soup.find_all('td', attrs={'class':'tleft2'})  

#values is the associated value
Values = soup.find_all('td', attrs={'class':'tleft'})  

In [60]:
# list the first 3 items 
print(commodity[0:3])
print(len(commodity))
print(Values[0:3])
print(len(Values))

[<td class="tleft2" valign="top">AMADUMBE</td>, <td class="tleft2" valign="top">APPLES</td>, <td class="tleft2" valign="top">ASPARAGUS</td>]
176
[<td class="tleft" valign="top">R5,120.00<br/>MTD: R36,956.00</td>, <td class="tleft" valign="top">19<br/>MTD: 514</td>, <td class="tleft" valign="top">236<br/>MTD: 2,504</td>]
704


In [61]:
# create an empty records list
# loop through the records list and obtain the various text pieces that provide the name of the commodity
# update/append the name to the records list
records = []  
for i in range(0, len(commodity)):  
    commodity_name = commodity[i].text
    records.append((commodity_name))


# go through the values list and get the various quantities

# 1 . first we do it for the value sold piece
    
value_sold = []
for i in range(0, len(Values),4):
        # obtain the text portion of the value
        value_sold_1 = Values[i].text
        # clean up the text portion and remove any , commas;
        value_sold_2 = value_sold_1.replace(',','')
        value_sold_3 = value_sold_2.replace('R','')
        value_sold.append((value_sold_3))

        
# 2 . we do it for the volume sold piece
volume_sold = []
for i in range(1, len(Values),4):
        # obtain the text portion of the value
        volume_sold_1 = Values[i].text
        # clean up the text portion and remove any , commas
        volume_sold_2 = volume_sold_1.replace(',','')
        volume_sold.append((volume_sold_2))

#3. we do it for the KG sold piece
kg_sold = []
for i in range(2, len(Values),4):
        # obtain the text portion of the value
        kg_sold_1 = Values[i].text
        # clean up the text portion and remove any , commas
        kg_sold_2 = kg_sold_1.replace(',','')
        kg_sold.append((kg_sold_2))
        
#4. we do it for the Quantity avalible within the market
volume_avil = []
for i in range(3, len(Values),4):
        # obtain the text portion of the value
        volume_avil_1 = Values[i].text
        # clean up the text portion and remove any , commas; also store number as floating point
        volume_avil_2 = float(volume_avil_1.replace(',',''))
        volume_avil.append((volume_avil_2))

In [62]:
print("the number of items in the various tables:")
print("Name of produce:" + str(len(records)))
print("Value Sold Count:" + str(len(value_sold)))
print("volume Sold Count:" + str(len(volume_sold)))
print("KG Sold Count:" + str(len(kg_sold)))
print("Volume Avalible Count:" + str(len(volume_avil)))

the number of items in the various tables:
Name of produce:176
Value Sold Count:176
volume Sold Count:176
KG Sold Count:176
Volume Avalible Count:176


In [63]:
# split the value_sold list up
# it uses the 'MTD: ' as the seperator
# the first array [0] gives us the value for today
# the second array [1] gives us the MTD value
value_sold_TD = [i.split('MTD: ', 1)[0] for i in value_sold]
value_sold_MTD = [i.split('MTD: ', 1)[1] for i in value_sold]


# split the volume_sold list up
volume_sold_TD = [i.split('MTD: ', 1)[0] for i in volume_sold]
volume_sold_MTD = [i.split('MTD: ', 1)[1] for i in volume_sold]

# split the kg_sold list up
kg_sold_TD = [i.split('MTD: ', 1)[0] for i in kg_sold]
kg_sold_MTD = [i.split('MTD: ', 1)[1] for i in kg_sold]


# Get the date of the last upload

In [64]:
#get the date of the last upload
soup.find_all('p')[1]

<p>This information is for <b>20 June 2018</b> </p>

In [65]:
date = soup.find_all('p')[1].text

In [66]:
#import the regex function
import re

# get the first digit and print it the string
print("First digit in the string date is : " + re.search(r'\d+', date).group())
# get the start position of the string
print("First digit in the string date starts @ position : " + str(re.search(r'\d+', date).start()))

First digit in the string date is : 20
First digit in the string date starts @ position : 24


In [67]:
date_str = date[re.search(r'\d+', date).start():]
print(date_str)

20 June 2018 


In [68]:
# convert string date to numeric python date

from datetime import datetime
from dateutil.parser import parse

In [69]:
parse(date_str)

datetime.datetime(2018, 6, 20, 0, 0)

In [70]:
# generate filename with leading zeros on month and date ie always 2 digits
fileout = str(parse(date_str).year) + str(parse(date_str).month).zfill(2)  + str(parse(date_str).day).zfill(2) 

In [71]:
fileout

'20180620'

## Create a pandas dataframe with all the information we require

In [72]:
# create a list from the filout value; this will be a list of Date that is XXX lenght long to allow it to merge correctly
# into the pandas dataframe
Date = [fileout] * len(records)


In [73]:
# create a dataframe based on lists
import pandas as pd  
df = pd.DataFrame(data=list(zip(records,value_sold_TD,value_sold_MTD,volume_sold_TD,
                                volume_sold_MTD,kg_sold_TD,kg_sold_MTD,volume_avil,Date)), 
                  columns=['Name', 'Value Sold (ZAR)', 'Value Sold (ZAR MTD)', 'Volume Sold',
                           'Volume sold (MTD)', 'KG Sold', 'KG Sold (MTD)','Volume Avalible','Date'])  

In [74]:
#convert numbers to float in dataframe and not character - object
df.iloc[:,1:-1]=df.iloc[:,1:-1].astype(float)

In [75]:
df.head()

Name  Value Sold (ZAR)  Value Sold (ZAR MTD)  Volume Sold  \
0   AMADUMBE            5120.0              36956.00         19.0   
1     APPLES         1234714.0           22014722.90      13754.0   
2  ASPARAGUS             300.0             200855.00          3.0   
3    ATCHARA               0.0                  0.00          0.0   
4   AVOCADOS          416167.0            6759677.22       7918.0   

   Volume sold (MTD)   KG Sold  KG Sold (MTD)  Volume Avalible      Date  
0              514.0     236.0         2504.0            127.0  20180620  
1           273368.0  169614.0      3156627.0         145196.0  20180620  
2              464.0       6.0         1400.0             66.0  20180620  
3                0.0       0.0            0.0             16.0  20180620  
4           139473.0   55071.0       893008.0          44221.0  20180620

# Write the dataframe to a text file with the correct date in the filename

In [76]:
import glob

In [77]:
# Give the filename you wish to save the file to
#path = 'C:\\Users\\atish\Documents\\Web Scrapping\\Data\\' + fileout + '.txt'
#path_1 = 'C:\\Users\\atish\\Google Drive\\Scripts\\Python\\Web Scrapping\\Data\\'  + fileout + '.txt'
path = 'Data\\'  + fileout + '.txt'
print(path)

Data\20180620.txt


In [78]:
# Use this function to search for any files which match your filename
files_present = glob.glob(path)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    df.to_csv(path , header=True, index=None, sep=',', mode='a')
else:
    print('WARNING: This file already exists!') 

# Do some analysis

In [79]:
df['ZAR/KG']=df['Value Sold (ZAR MTD)']/df['KG Sold (MTD)']

In [80]:
df.sort_values(by='ZAR/KG', ascending=False)

Name  Value Sold (ZAR)  Value Sold (ZAR MTD)  \
109                     NUTS               0.0                800.00   
50                  CROUTONS               0.0                 75.20   
152                   SNACKS               0.0                 90.00   
149                    SALAD             240.0               1200.00   
102              MIXED HERBS               0.0               2037.00   
115                  OREGANO               0.0               1856.00   
43                    CHIVES               0.0                785.00   
23               BLUEBERRIES               0.0              27200.00   
166                    THYME             179.0               5755.99   
22              BLACKBERRIES            7200.0              48144.00   
2                  ASPARAGUS             300.0             200855.00   
147                 ROSEMARY             475.0              13127.00   
160              SWEET BASIL            1728.0              15211.00   
40                  CHERRIES           26930.0             693259.00   
143              RASPBERRIES           26160.0             225271.67   
94                  MARJORAM               0.0               2329.00   
172               WATERCRESS               0.0                426.00   
157     STARFRUIT(CARAMBOLA)            5000.0              29660.00   
53              CURRY LEAVES            1645.0              32513.00   
10                 BABY LEAF             480.0              11220.00   
62                      FIGS            3960.0              68170.00   
146                   ROCKET             741.0              19833.00   
165                TAMARILLO            1280.0               7312.00   
16                     BASIL             640.0               1940.00   
158             STRAWBERRIES          310680.0            4138605.00   
86               LEMON GRASS             225.0                600.00   
69          GINGER PROCESSED               0.0               1587.72   
126        PECAN NUT (SHELL)            1720.0              58638.00   
123  PEANUTS RAW / UNSHELLED               0.0                120.00   
150              SALTED NUTS               0.0              10200.00   
..                       ...               ...                   ...   
26           BROCCOLI LEAVES             140.0                140.00   
48                  COCONUTS            7120.0              77600.00   
87                    LEMONS          222700.0            2093698.17   
170                   TURNIP            3120.0             110392.89   
155                  SPINACH           64906.0            1371914.13   
128               PINEAPPLES           58468.0            2280737.00   
42       CHINESE  VEGETABLES            1453.0             105869.80   
112                   ONIONS         2797932.5           45930913.80   
21                  BEETROOT          205982.0            4431858.00   
33                   CARROTS          559964.0           11830645.00   
114                  ORANGES          978897.0           16043417.00   
134                 POTATOES         4187786.5           76608243.10   
77          HUBBARD SQUASHES          190400.0            2296812.16   
163             SWEET POTATO          151227.0            2958364.50   
30                   CABBAGE          181822.0            5228064.83   
29                 BUTTERNUT          499357.0            6600589.00   
92                MAIZE MEAL               0.0                190.00   
103                  MORINGA               0.0                100.00   
63                  FIREWOOD            5630.0              62803.00   
130           PKS BLUE QUEEN           45800.0            1242134.00   
137                 PUMPKINS           70650.0             507629.00   
3                    ATCHARA               0.0                  0.00   
78            JACK BE LITTLE               0.0                  0.00   
83                    KUMATO               0.0                  

In [81]:
df[df.Name.str.contains('GARLIC', case=False, regex=False)]

Name  Value Sold (ZAR)  Value Sold (ZAR MTD)  \
51  CRUSHED GINGER/GARLIC           1257.00              20908.20   
64                 GARLIC         208540.00            2457227.00   
65       GARLIC PROCESSED           5109.81              85503.55   

    Volume Sold  Volume sold (MTD)  KG Sold  KG Sold (MTD)  Volume Avalible  \
51          5.0               91.0     30.0          428.0             52.0   
64        499.0             9998.0   4210.0        64709.0           9384.0   
65         26.0              461.0    135.0         2221.0            282.0   

        Date     ZAR/KG  
51  20180620  48.850935  
64  20180620  37.973497  
65  20180620  38.497771